## import

In [120]:
import pandas as pd
from smds import SMDS
import IPython.display as ipd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import shuffle
from utils import min_max_normalization, TestModel
from keras import models, layers

## data

In [121]:
# load labels
data = pd.read_csv("./data/gtzan/features_30_sec.csv")
genres = data[["filename", "label"]]
Y = data["label"]

In [122]:
# load preprocessed data
smd = SMDS()
smd_data = smd.load("./data/smds/smds_13.csv")
smd_tempo = smd.load_tempo("./data/smds/smds_13_tempo.csv")
ipd.display(smd_data)
ipd.display(smd_tempo)

n_mfcc                           0                                             \
stat                      kurtosis       max       mean     median        min   
id  filename        tick                                                        
0   blues.00000.wav 0    -0.921429 -0.602207  -4.116866  -4.102283  -7.520702   
                    1    -1.034314 -1.877029  -7.598314  -7.186878 -13.353165   
                    2    -1.572501 -0.969917  -4.778881  -5.134876  -8.422367   
                    3    -0.464004 -6.557148 -19.352211 -13.589458 -44.356602   
                    4    -1.543346 -3.608807  -9.907966 -11.096891 -15.579109   
...                            ...       ...        ...        ...        ...   
999 rock.00099.wav  125  -1.668841 -6.418808  -8.240566  -8.720350  -9.642128   
                    126  -0.565443 -6.335140  -7.947320  -8.316345  -8.830395   
                    127  -1.263555 -7.196236  -7.737393  -7.752090  -8.294815   
                    128  -0.988667 -6.559055  -7.195686  -7.254095  -7.784954   
                    129   0.269860  0.000000  -1.463675   0.000000  -7.539124   

n_mfcc                                                1                        \
stat                          skew        std  kurtosis        max       mean   
id  filename        tick                                                        
0   blues.00000.wav 0     0.075946   2.036903  0.772538  -5.987366  -7.507081   
                    1    -0.174034   3.668512  0.112355  -4.566875  -5.301490   
                    2     0.114478   2.887686 -1.140482  -5.294666  -6.178790   
                    3    -1.017900  13.109632 -0.113993  -5.659710  -6.246984   
                    4     0.186728   4.365551 -0.444909  -5.034054  -6.183269   
...                            ...        ...       ...        ...        ...   
999 rock.00099.wav  125   0.288957   1.312025 -1.685759 -11.284740 -16.591238   
                    126   0.939037   0.814353  0.135143 -10.887753 -13.876005   
                    127   0.007978   0.383713  0.305037 -11.849773 -13.116172   
                    128   0.295401   0.383454 -0.891905 -12.233089 -13.124200   
                    129  -1.504257   2.929015  0.256183   0.000000  -2.852111   

n_mfcc                    ...         11                             12  \
stat                      ...        min      skew        std  kurtosis   
id  filename        tick  ...                                             
0   blues.00000.wav 0     ... -60.223705 -1.252348   8.094678 -0.558997   
                    1     ... -39.332470 -0.483045   3.570479  0.491824   
                    2     ... -44.687153 -0.152334   6.106122 -1.602251   
                    3     ... -28.273235 -0.827198   2.190228  0.502444   
                    4     ... -46.660244 -0.598231   8.158568  1.594014   
...                       ...        ...       ...        ...       ...   
999 rock.00099.wav  125   ... -32.267956  0.229043   2.567013 -1.421341   
                    126   ... -35.265717  0.395222   3.551404  0.340641   
                    127   ... -26.069447 -0.175981   0.682385 -0.081579   
                    128   ... -28.157679 -0.633861   1.239221 -0.325619   
                    129   ... -26.403402 -1.503891  10.270678  0.283285   

n_mfcc                                                                \
stat                            max       mean     median        min   
id  filename        tick                                               
0   blues.00000.wav 0    -31.908342 -40.000139 -39.760805 -48.561104   
                    1    -28.242262 -34.864012 -33.765718 -47.739788   
                    2    -27.690639 -31.491880 -30.364546 -36.691635   
                    3    -32.313374 -43.576982 -40.048849 -72.330002   
                    4    -29.337311 -42.734724 -41.277388 -72.304787   
...                             ...        ...        ...        ...   
999 rock.00099.wav  125  -23.87

,filename,tempo
0,blues.00000.wav,123.046875
1,blues.00001.wav,107.666016
2,blues.00002.wav,161.499023
3,blues.00003.wav,172.265625
4,blues.00004.wav,135.999178
...,...,...
995,rock.00095.wav,103.359375
996,rock.00096.wav,117.453835
997,rock.00097.wav,129.199219
998,rock.00098.wav,112.347147


In [123]:
def prepare_data(data: pd.DataFrame):
    grouped = data.groupby(by="id")
    
    cnn_data = [i for _, i in grouped]
    # cnn_data = np.expand_dims(cnn_data, axis=-1)
    return np.array(cnn_data)

def prepare_tempo_data(data: pd.DataFrame):
    tempo_data = [i for i in data["tempo"]]
    return np.array(tempo_data)
    

In [124]:
X_STAT = prepare_data(smd_data)
X_TEMPO = prepare_tempo_data(smd_tempo)
X_STAT, X_TEMPO, Y = shuffle(X_STAT, X_TEMPO, Y, random_state=1)
X_STAT.shape, X_TEMPO.shape, Y.shape
# 1000  songs
# 130   splits
# 91    statistical features (13 tracks * 7 features)
# 1     for CNN channel

((1000, 130, 91), (1000,), (1000,))

In [125]:
# 將 NaN 替換為 0，無窮大替換為合理的值

X_STAT = np.nan_to_num(X_STAT, nan=0.0, posinf=1.0, neginf=-1.0)
print(np.isnan(X_STAT).any())
print(np.isinf(X_STAT).any()) 

X_TEMPO = np.nan_to_num(X_TEMPO, nan=0.0, posinf=1000.0, neginf=0.0)
print(np.isnan(X_TEMPO).any())
print(np.isinf(X_TEMPO).any())

False
False
False
False


## Model 1

In [136]:
def build_encoder(input_shape: tuple, output_shape: int):
    # input_shape: (spilts, statistifics)
    input_layer = layers.Input(shape=input_shape)
    x = layers.Conv1D(64, kernel_size=3, activation='sigmoid', padding='same')(input_layer)
    x = layers.Conv1D(128, kernel_size=3, activation='sigmoid', padding='same')(x)
    x = layers.Flatten()(x)
    features = layers.Dense(output_shape, activation='sigmoid')(x)
    return models.Model(inputs=input_layer, outputs=features, name="encoder")

encoder = build_encoder((X_STAT.shape[1], X_STAT.shape[2]), 16)
encoder.summary()
encoder.output_shape

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_87 (InputLayer)     │ (None, 130, 91)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_94 (Conv1D)              │ (None, 130, 64)        │        17,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_95 (Conv1D)              │ (None, 130, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_30 (Flatten)            │ (None, 16640)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (None, 16)             │       266,256 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 308,496 (1.18 MB)

 Trainable params: 308,496 (1.18 MB)

 Non-trainable params: 0 (0.00 B)

(None, 16)

In [137]:
def build_decoder(input_shape: tuple, output_shape: tuple):
    input_layer = layers.Input(shape=input_shape)
    x = layers.Dense(output_shape[0] * 128, activation='sigmoid')(input_layer)
    x = layers.Reshape((output_shape[0], 128))(x)
    x = layers.Conv1DTranspose(64, kernel_size=3, activation='sigmoid', padding='same')(x)
    reconstructed = layers.Conv1D(output_shape[1], kernel_size=3, activation='sigmoid', padding='same')(x)
    return models.Model(inputs=input_layer, outputs=reconstructed, name='decoder')

# input_shape     -> int
# (input_shape)   -> int
# (input_shape, ) -> tuple
decoder = build_decoder((16, ), (X_STAT.shape[1], X_STAT.shape[2]))
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_88 (InputLayer)     │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_58 (Dense)                │ (None, 16640)          │       282,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_27 (Reshape)            │ (None, 130, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_27             │ (None, 130, 64)        │        24,640 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_96 (Conv1D)              │ (None, 130, 91)        │        17,563 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 325,083 (1.24 MB)

 Trainable params: 325,083 (1.24 MB)

 Non-trainable params: 0 (0.00 B)

In [138]:
def build_preprocessor(data):
    normalizer = layers.Normalization()
    normalizer.adapt(data) # fit
    return normalizer

In [147]:
def build_autoencoder(input_shape, latent_dim, normalizer):
    # latent_dim: encoder output shape
    encoder = build_encoder(input_shape, latent_dim)
    decoder = build_decoder((latent_dim, ), input_shape)
    
    input_layer = layers.Input(shape=input_shape)
    normalized_data = normalizer(input_layer)
    features = encoder(normalized_data)
    reconstructed = decoder(features)
    
    autoencoder = models.Model(inputs=input_layer, outputs=reconstructed, name="autoencoder")
    return autoencoder

normalizer = build_preprocessor(X_STAT)
autoencoder = build_autoencoder(X_STAT.shape[1:], 16, normalizer)
autoencoder.summary(expand_nested=True)
    

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_100 (InputLayer)    │ (None, 130, 91)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization_10                │ (None, 130, 91)        │           183 │
│ (Normalization)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ (None, 16)             │       308,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ input_layer_98             │ (None, 130, 91)        │             0 │
│ (InputLayer)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ conv1d_106 (Conv1D)        │ (None, 130, 64)        │        17,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ conv1d_107 (Conv1D)        │ (None, 130, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ flatten_34 (Flatten)       │ (None, 16640)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_65 (Dense)           │ (None, 16)             │       266,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 130, 91)        │       325,083 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ input_layer_99             │ (None, 16)             │             0 │
│ (InputLayer)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_66 (Dense)           │ (None, 16640)          │       282,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ reshape_31 (Reshape)       │ (None, 130, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ conv1d_transpose_31        │ (None, 130, 64)        │        24,640 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ conv1d_108 (Conv1D)        │ (None, 130, 91)        │        17,563 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 633,762 (2.42 MB)

 Trainable params: 633,579 (2.42 MB)

 Non-trainable params: 183 (736.00 B)

In [140]:
autoencoder.compile(optimizer="adam", loss="mse")
hist = autoencoder.fit(X_STAT, X_STAT, epochs=100, batch_size=20, validation_split=0.1)

Epoch 1/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 556.5461 - val_loss: 529.5715
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 548.4854 - val_loss: 529.4151
Epoch 3/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 538.7850 - val_loss: 529.3820
Epoch 4/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 548.9574 - val_loss: 529.3655
Epoch 5/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 553.9662 - val_loss: 529.3557
Epoch 6/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 541.9745 - val_loss: 529.3494
Epoch 7/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 540.8611 - val_loss: 529.3450
Epoch 8/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 545.1695 - val_loss: 529.3420
Epoch 9/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 556.2087 - val_loss: 529.3397
Epoch 10/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 549.1368 - val_loss: 529.3380
Epoch 11/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 544.9760 - val_loss: 529.3366
Epoch 12/100
45/45 

In [141]:
encoder = autoencoder.get_layer("encoder")
single_sample = np.expand_dims(X_STAT[0], axis=0)
single_sample.shape


(1, 130, 91)

In [142]:

encoder.predict(single_sample)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


array([[1.000000e+00, 1.000000e+00, 4.516795e-12, 1.000000e+00,
        1.000000e+00, 1.000000e+00, 1.000000e+00, 1.000000e+00,
        1.000000e+00, 1.000000e+00, 1.000000e+00, 1.000000e+00,
        1.000000e+00, 1.000000e+00, 1.000000e+00, 1.000000e+00]],
      dtype=float32)

In [144]:
normalized_data = normalizer(X_STAT).numpy()
print("Normalized Data Range:", normalized_data.min(), normalized_data.max())

Normalized Data Range: -19.192059 14.532973
